In [1]:
import re
import string


def preprocess_document(doc_bytes):
    """
    Args:
        :param doc_bytes: The document to be processed.

    Returns:
        :rtype: str
    """
    try:
        with open (doc_bytes, "r") as my_file:
            text = my_file.read()
            
            return text
        
    except:
        return False
    # Additional cleaning steps (optional)

In [16]:
def is_question_format(question):
    """
    Validate the format of the question.

    Args:
        question (str): The user's input string.

    Returns:
        bool: True if the input string is a valid question, False otherwise.
    """
    try:
        # Define a regex pattern to match common question structures
        question_pattern = r"^[^\W\d_]+\s*(?:\s+[^\W\d_]+)*\s*\?$"

        # Check if the input string matches the question pattern
        if re.match(question_pattern, question.strip()):
            return True
        else:
            return False
    except Exception as e:
        # Handle any errors gracefully and return False
        # print(f"Error validating question format: {e}")
        return False

if not is_question_format("dasdf"):
    print("Enter a valid question format")

Enter a valid question format


In [5]:
# Main logic for answering questions
from transformers import pipeline
# Load the pre-trained question answering model
qa_model = pipeline("question-answering", model="timpal0l/mdeberta-v3-base-squad2")


def answer_question(doc_bytes, question):
    # clean_text = preprocess_document(doc_bytes)
    # If not found in the database, use the AI model to answer the question
    context = preprocess_document(doc_bytes)
    answer = qa_model(question=question, context=context)["answer"]

    return answer

In [6]:
# User interface setup using Gradio

import gradio as gr

interface = gr.Interface(
    fn=answer_question,
    inputs=[gr.File(label="Upload Document"), "text"],
    outputs="text",
    title="Question Answering System",
)
interface.launch()


Running on local URL:  http://127.0.0.1:7861

To create a public link, set `share=True` in `launch()`.


Traceback (most recent call last):
  File "c:\Users\BA\anaconda3\lib\site-packages\gradio\queueing.py", line 495, in call_prediction
    output = await route_utils.call_process_api(
  File "c:\Users\BA\anaconda3\lib\site-packages\gradio\route_utils.py", line 235, in call_process_api
    output = await app.get_blocks().process_api(
  File "c:\Users\BA\anaconda3\lib\site-packages\gradio\blocks.py", line 1627, in process_api
    result = await self.call_function(
  File "c:\Users\BA\anaconda3\lib\site-packages\gradio\blocks.py", line 1173, in call_function
    prediction = await anyio.to_thread.run_sync(
  File "c:\Users\BA\anaconda3\lib\site-packages\anyio\to_thread.py", line 31, in run_sync
    return await get_asynclib().run_sync_in_worker_thread(
  File "c:\Users\BA\anaconda3\lib\site-packages\anyio\_backends\_asyncio.py", line 937, in run_sync_in_worker_thread
    return await future
  File "c:\Users\BA\anaconda3\lib\site-packages\anyio\_backends\_asyncio.py", line 867, in run
    re